In [1]:
def PCA(Y,X,varExplained):
        
    # On transforme X en une matrice de moyenne = 0 et variance = 1
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)
    nbrOfVar = X.shape[1]

    # On calcule la matrice de covariance
    E = np.dot(X_std.T,X_std)/(nbrOfVar-1)

    # On calcule nos Valeurs et Vecteurs propres
    eig_val, eig_vec = np.linalg.eig(E)
    eig_val = eig_val.real
    eig_vec = eig_vec.T # pour les ordonner par range

    # On calcule la contribution de chaque valeur propre a la variance total
    eig_val_contribution = eig_val/np.sum(eig_val)

    # On cree une liste d'index de 0 au nombre de valeurs propres
    rangeIndex = np.array(range(len(eig_val_contribution)))

    # On cree un tableau indexe de nos valeurs propres et leur contribution a la variance
    eig_val_table = np.array(([rangeIndex,eig_val,eig_val_contribution])).T

    # On ordonne en ordre decroissant de contribution le tableau
    eig_val_table_ordered = eig_val_table[(-eig_val_table[:,2]).argsort()]
    
    # On rejoint la quantite de variance souhaitee
    varCumul = 0.0
    varIndex = 0
    for val in eig_val_table_ordered:
        varIndex += 1
        varCumul += val[2]
        if(varCumul >= varExplained):
            break
    
    # On recupere les index des valeurs propres que nous garderont
    indexKept = eig_val_table_ordered[:,0][:varIndex]

    # Our reduced regressor
    K_vect = eig_vec[indexKept.astype(int),:].T

    # Our principal components
    Z = np.dot(X_std,K_vect)

    # Our Regression Coefficiants
    B_partial = np.dot(np.linalg.inv(np.dot(Z.T,Z)),Z.T)
    B = np.dot(B_partial,Y)

    # We calculate the intercept
    uY = np.mean(Y)

    # We add the intercept
    Y_new = np.dot(Z,B) + uY

    # Clip to min 0
    Y_new = Y_new.clip(min=0)
    
    return Y_new, B, K_vect, uY


In [2]:
def PCA_trained(X,B,K_vect,uY):
    
    X_std = StandardScaler().fit_transform(X)
    
    # Our principal components
    Z = np.dot(X_std,K_vect)
    
    # We add the intercept
    Y_new = np.dot(Z,B) + uY
    
    # Clip to min 0
    Y_new = Y_new.clip(min=0)

    return Y_new
    

In [3]:
def estimateCol(trainSet,testSet,colName,varExplained):
    
    # On definit notre variable d'interet
    Y = np.array(trainSet[colName])

    # On definit notre vecteur de variables explicatives
    X = np.array(trainSet.drop([colName], axis=1))

    # We train our model
    [Y_new, B, K_vect, uY] = PCA(Y,X,varExplained)
    
    X_test = np.array(testSet.drop([colName], axis=1))    
    Y_test = PCA_trained(X_test,B,K_vect,uY)
    
    return Y_test

In [4]:
df_test_estimated = pd.DataFrame([df_test.NA_Sales,df_test.Global_Sales]).copy().T


df_test_estimated['NA_Sales'] = estimateCol(df_train,df_test,'NA_Sales',0.999)
df_test_estimated['Global_Sales'] = estimateCol(df_train,df_test,'Global_Sales',0.999)

# On definit notre vecteur de variables explicatives
X1 = np.array(trainSet.drop(['Global_Sales'], axis=1))
X2 = np.array(trainSet.drop(['NA_Sales'], axis=1))

Y1 = df_test_estimated.Global_Sales.values
Y2 = df_test_estimated.NA_Sales.values

print("Global R2_prev : %.3f" % (R2_prev(Y1,np.dot(X1,B1),X)))
print("NA R2_prev : %.3f" % (R2_prev(Y2,np.dot(X2,B2),X)))

NameError: name 'pd' is not defined